<a href="https://colab.research.google.com/github/selmakurtovic4/HeartAttack/blob/main/HeartAttack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
!git clone https://github.com/selmakurtovic4/HeartAttack.git
os.chdir('/content/HeartAttack')

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, MinMaxScaler


In [3]:
df=pd.read_csv('heart.csv')

In [4]:
summary_stats = df.describe()

In [18]:
summary_stats

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


Kolone koje imaju outliere su: chol,trtbps, fbs, thalachh, oldpeak, caa, thall


---


Nemaju: age,cp,sex, restecg, exng, slp

In [ ]:
# Calculate quartiles and IQR
Q1 = df['thall'].quantile(0.25)
Q3 = df['thall'].quantile(0.75)
IQR = Q3 - Q1

# Identify potential outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Use parentheses to group conditions properly
outliers = df[(df['thall'] < lower_bound) | (df['thall'] > upper_bound)]

print("Potential outliers:")
print(outliers)


In [47]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]

scalling

In [ ]:
outlier_cols = ["chol","trtbps","fbs","thalachh","oldpeak","caa","thall"]
normal_cols=["age","cp","sex","restecg","exng","slp"]
robust_scaler = RobustScaler()
minmax_scaler = MinMaxScaler()
X[outlier_cols] = robust_scaler.fit_transform(X[outlier_cols])
X[normal_cols] = minmax_scaler.fit_transform(X[normal_cols])

In [51]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2,random_state=42)

Model

In [79]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [76]:
def create_model(input_shape):
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    learning_rate = 0.01
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy', 'val_accuracy'])
    return model

In [78]:
input_shape = X_train.shape[1]
model = create_model(input_shape)

In [77]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [ ]:
batch_size = 32
epochs = 50

history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[early_stopping])

train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_accuracy) + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracies')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [75]:
test_loss, test_accuracy = model.evaluate(X_test, Y_test)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

2/2 [==============================] - 0s 7ms/step - loss: 0.9946 - accuracy: 0.8525
Test Loss: 0.9946138858795166
Test Accuracy: 0.8524590134620667
